In [2]:
import pandas as pd
import numpy as np
import re

Global variables


In [3]:
leagues_prefixs = ["SA1", "SA2L"]
clubs_list = []
champions_list = []

Transform functions


In [4]:
# Function to transform attendences DataFrame

def attendances_tranform():
    df_f = pd.read_csv("./RAW/ArabicSoccer/Attendances/Arabic_Attendances_2007_2023.csv")

    # In column "#" where the value is NaN is duplicate row
    # So I decide to remove using comparision "['#'] >= 1"
    df = df_f[df_f['#'] >= 1]
    
    # Reorganize and select columns
    df = df[["Stadium", "Capacity", "Spectators",
             "Average", "Matches", "sold out", "Club", "Year"]]

    # Pattern to separate the stadium name do club name
    pattern = "\B[ANH][Llab][-]?[a-zA-Z]+(?:-[a-zA-Z]+)?\s?[SFClub]{1,4}"

    # I use these if's because I can't catch these words in my pattern
    for i in range(len(df)):
        if "Al-Ansar" in df.iloc[i, 0]:
            df.iloc[i, 6] = "Al-Ansar"
            df.iloc[i, 0] = "Prince Mohammed bin Abdul Aziz Stadium"
        elif "Ohod Club" in df.iloc[i, 0]:
            df.iloc[i, 6] = "Ohod Club"
            df.iloc[i, 0] = "Prince Mohammed bin Abdul Aziz Stadium"
        elif "Damac FC" in df.iloc[i, 0]:
            df.iloc[i, 6] = "Damac FC"
            df.iloc[i, 0] = "Damac Club Stadium"
        else:
            # In re.findall() I use regex to catch teams name
            # King Abdullah Sports CityAl-Ittihad Club ----> ['Al-Ittihad Club']
            df.iloc[i, 6] = re.findall(pattern=pattern, string=df.iloc[i, 0])[0]

            # Using re.sub() I can replace the clubs names in String to "", leaving the line with just the name of the stadium
            df.iloc[i, 0] = re.sub(pattern=pattern, string=df.iloc[i, 0], repl="")

    # Reset the index, since there was a change in the DataFrame rows.
    df = df.reset_index()

    # Create a id column
    df["index"] = df.index + 1
    df = df.rename(columns={"index": "id"})

    # Changing "Matches" column to Integer
    df['Matches'] = pd.to_numeric(df['Matches'], downcast='integer', errors='coerce')

    return df

In [5]:
# Function to transform champions DataFrame

def champions_list_transform():
    for i in range(len(leagues_prefixs)):
        df = pd.read_csv(f"./RAW/ArabicSoccer/Champions/{leagues_prefixs[i]}_Champions.csv")
        df = df[["Season", "Club", "League"]]
        df.rename(columns={"Club": "Clubs"}, inplace=True)
        df["id"] = df.index + 1
        df = df[["id", "Clubs", "League", "Season"]]
        # The loop is based in the arabic leagues
        # The i == 1 correspond to the champions of SA1, and the i == 2 of SA2L. The delimiters are to capture all champions from each league from the 2007 to 2023
        if i == 2:
            df = df[:15]
            champions_list.append(df)
        else:
            df = df[:16]
            champions_list.append(df)

In [6]:
# Function to transform teams DataFrames

def teams_transform():
    for y in range(len(leagues_prefixs)):
        df = pd.read_csv(f"./RAW/ArabicSoccer/Teams/{leagues_prefixs[y]}_teams_2023.csv")
        df = df[["Club.1", "name", "Squad", "ø age","Foreigners", "ø market value", "League"]]
        
        # Rename columns
        df.rename(columns={"Club.1": "Clubs", "name": "Players", "Squad": "avg_age", "ø age": "Foreigners",
                  "Foreigners": "market_value(€)", "ø market value": "total_market_value(€)"}, inplace=True)

        # I used regex to replace special characters with empty strings, to transform into an integer later
        for i, row in df.iterrows():
            df.iloc[i, 4] = re.sub(pattern='[€mkK]', string=row["market_value(€)"], repl="")
            df.iloc[i, 5] = re.sub(pattern='[€mkK]', string=row["total_market_value(€)"], repl="")

        # Removes the last column, which is an aggregation of data, present in the original table
        df = df[:-1]
        df["id"] = df.index + 1

        # Changing columns dtypes
        df["Foreigners"] = pd.to_numeric(df["Foreigners"], downcast='integer', errors='coerce')
        df['market_value(€)'] = pd.to_numeric(df['market_value(€)'], downcast='float', errors='coerce')
        df['total_market_value(€)'] = pd.to_numeric(df['total_market_value(€)'], downcast='float', errors='coerce')

        # Round columns
        df['market_value(€)'] = df['market_value(€)'].round()
        df['total_market_value(€)'] = df['total_market_value(€)'].round()

        # Change the columns order
        df = df[["id", "Clubs", "Players", "avg_age", "Foreigners","market_value(€)", "total_market_value(€)", "League"]]

        clubs_list.append(df)

Transfers data from Kaggle

In [7]:
# This CSV is from Kaggle

def transfers_spl():
  df_og = pd.read_csv("RAW\ArabicSoccer\Transfers\saudi-pro-league-transfers.csv")
  df = df_og[["club_name", "country","player_name", "age", "position", "club_involved_name", "club_involved_country","transfer_movement","transfer_period","year"]]
  df.loc[:, ["club_involved_name", "club_involved_country"]] = df.loc[:, ["club_involved_name", "club_involved_country"]].replace("Unknown", None)
  df.loc[:, ["club_involved_name", "club_involved_country"]] = df.loc[:, ["club_involved_name", "club_involved_country"]].replace("Without Club", "Free Agent")
  df.loc[:, ['age', 'year']] = df.loc[:, ['age', 'year']].fillna(0)
  df.loc[:, ['age', 'year']] = df.loc[:, ['age', 'year']].astype(np.int64)

  df["id"] = df.index + 1

  return df

Performing the transformation

In [8]:
df_attendances = attendances_tranform()

teams_transform()
df_teams = pd.concat(clubs_list)

champions_list_transform()
df_champions = pd.concat(champions_list)

df_transfers = transfers_spl()

C:\Users\mathe\AppData\Local\Temp\ipykernel_5456\3249917773.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["id"] = df.index + 1


Dataframes

In [9]:
df_attendances.head()

,id,Stadium,Capacity,Spectators,Average,Matches,sold out,Club,Year
0,1,King Abdullah Sports City,62.345,27.0,27.0,1,-,Al-Ittihad Club,2007
1,2,King Fahd International Stadium,68.752,0.0,0.0,0,-,Al-Hilal SFC,2007
2,3,Prince Mohamed bin Fahd Stadium,35.000,0.0,0.0,0,-,Al-Ettifaq FC,2007
3,4,Al-Hazem Club Stadium,8.000,0.0,0.0,0,-,Al-Hazem SC,2007
4,5,Prince Faisal bin Fahad Stadium,22.500,0.0,0.0,0,-,Al-Shabab FC,2007


In [10]:
df_teams.head()

,id,Clubs,Players,avg_age,Foreigners,market_value(€),total_market_value(€),League
0,1,Al-Hilal SFC,28.0,28.0,9,9.0,258.0,SA1
1,2,Al-Ahli SFC,29.0,25.6,9,7.0,201.0,SA1
2,3,Al-Nassr FC,27.0,28.1,9,7.0,196.0,SA1
3,4,Al-Ittihad Club,32.0,27.2,10,4.0,133.0,SA1
4,5,Al-Shabab FC,25.0,27.4,9,3.0,69.0,SA1


In [11]:
df_champions.head()

,id,Clubs,League,Season
0,1,Al-Ittihad Club,Yelo League,22/23
1,2,Al-Hilal SFC,Yelo League,21/22
2,3,Al-Hilal SFC,Yelo League,20/21
3,4,Al-Hilal SFC,Yelo League,19/20
4,5,Al-Nassr FC,Yelo League,18/19


In [12]:
df_transfers.head()

,club_name,country,player_name,age,position,club_involved_name,club_involved_country,transfer_movement,transfer_period,year,id
0,Al-Hilal SFC,Saudi Arabia,Khaled Aziz,18.0,Defensive Midfield,None,None,in,Summer,2000,1
1,Al-Hilal SFC,Saudi Arabia,Sami Al-Jaber,27.0,Centre-Forward,Wolves,England,out,Summer,2000,2
2,Al-Hilal SFC,Saudi Arabia,Fahad Al-Ghesheyan,26.0,Right Winger,Al-Nassr,Saudi Arabia,out,Summer,2000,3
3,Al-Hilal SFC,Saudi Arabia,Saleh Al-Salloumi,37.0,Goalkeeper,Retired,Retired,out,Summer,2000,4
4,Al-Hilal SFC,Saudi Arabia,Elijah Litana,29.0,Centre-Back,Retired,Retired,out,Summer,2000,5


Transform DataFrames in CSV file and send to Trusted Zone.

In [13]:
df_attendances.to_csv(f"./TRUSTED/ArabicSoccer/Attendances/Attendances_SPL_2007_2023.csv", encoding='utf-8', index=False)
df_teams.to_csv(f"./TRUSTED/ArabicSoccer/Teams/Teams_Arabia.csv", encoding='utf-8', index=False)
df_champions.to_csv(f"./TRUSTED/ArabicSoccer/Champions/Champions_Arabia_2007_2023.csv", encoding='utf-8', index=False)
df_transfers.to_csv(f"./TRUSTED/ArabicSoccer/Transfers/Champions_Arabia_2007_2023.csv", encoding='utf-8', index=False)